# Chapter 1: Mathematical Foundations and Implementation

## Building Math Libraries from Scratch

This notebook covers:
- Vector implementation (Vec2, Vec3, Vec4)
- Matrix implementation (Matrix3x3, Matrix4x4)
- Coordinate systems
- Transformation matrices
- Rotation representations (Euler angles, quaternions)

**Key References:** Marschner & Shirley Ch. 2, 5-6, Gambetta Ch. 1

---

## Setup and Imports

In [ ]:
import numpy as np
import math
from typing import Union, Tuple

# For visualization
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print("Libraries imported successfully!")

---

## 1. Vector Implementation

We'll build vector classes from scratch without using numpy for the core operations.

In [ ]:
class Vec3:
    """3D Vector implementation from scratch"""
    
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    # Addition
    def __add__(self, other: 'Vec3') -> 'Vec3':
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    # Subtraction
    def __sub__(self, other: 'Vec3') -> 'Vec3':
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    # Scalar multiplication
    def __mul__(self, scalar: float) -> 'Vec3':
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __rmul__(self, scalar: float) -> 'Vec3':
        return self.__mul__(scalar)
    
    # Scalar division
    def __truediv__(self, scalar: float) -> 'Vec3':
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    # Negation
    def __neg__(self) -> 'Vec3':
        return Vec3(-self.x, -self.y, -self.z)
    
    # Dot product
    def dot(self, other: 'Vec3') -> float:
        """Compute dot product: v · u = vx*ux + vy*uy + vz*uz"""
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    # Cross product
    def cross(self, other: 'Vec3') -> 'Vec3':
        """Compute cross product: v × u"""
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    # Length (magnitude)
    def length(self) -> float:
        """Compute vector length: ||v|| = sqrt(x² + y² + z²)"""
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)
    
    def length_squared(self) -> float:
        """Compute squared length (faster, no sqrt)"""
        return self.x**2 + self.y**2 + self.z**2
    
    # Normalization
    def normalize(self) -> 'Vec3':
        """Return normalized vector (unit length)"""
        length = self.length()
        if length > 0:
            return self / length
        return Vec3(0, 0, 0)
    
    def normalize_self(self) -> None:
        """Normalize this vector in-place"""
        length = self.length()
        if length > 0:
            self.x /= length
            self.y /= length
            self.z /= length
    
    # String representation
    def __repr__(self) -> str:
        return f"Vec3({self.x:.4f}, {self.y:.4f}, {self.z:.4f})"
    
    # Array-like access
    def __getitem__(self, index: int) -> float:
        if index == 0: return self.x
        elif index == 1: return self.y
        elif index == 2: return self.z
        else: raise IndexError("Vec3 index out of range")
    
    def to_list(self) -> list:
        return [self.x, self.y, self.z]

# Test Vec3
v1 = Vec3(1, 2, 3)
v2 = Vec3(4, 5, 6)

print("Vector Tests:")
print(f"v1 = {v1}")
print(f"v2 = {v2}")
print(f"v1 + v2 = {v1 + v2}")
print(f"v1 - v2 = {v1 - v2}")
print(f"v1 · v2 = {v1.dot(v2)}")
print(f"v1 × v2 = {v1.cross(v2)}")
print(f"||v1|| = {v1.length():.4f}")
print(f"normalize(v1) = {v1.normalize()}")

### Vector Utilities

In [ ]:
def lerp(v1: Vec3, v2: Vec3, t: float) -> Vec3:
    """Linear interpolation between two vectors"""
    return v1 * (1 - t) + v2 * t

def reflect(v: Vec3, n: Vec3) -> Vec3:
    """Reflect vector v about normal n"""
    return v - n * (2 * v.dot(n))

def refract(v: Vec3, n: Vec3, eta: float) -> Vec3:
    """Refract vector v through surface with normal n and index of refraction eta"""
    dt = v.dot(n)
    discriminant = 1.0 - eta * eta * (1.0 - dt * dt)
    if discriminant > 0:
        return (v - n * dt) * eta - n * math.sqrt(discriminant)
    else:
        # Total internal reflection
        return reflect(v, n)

# Test utilities
v = Vec3(1, -1, 0).normalize()
n = Vec3(0, 1, 0)
print(f"\nReflection test:")
print(f"Incident: {v}")
print(f"Normal: {n}")
print(f"Reflected: {reflect(v, n)}")

### Gram-Schmidt Orthogonalization

In [ ]:
def gram_schmidt(v1: Vec3, v2: Vec3, v3: Vec3) -> Tuple[Vec3, Vec3, Vec3]:
    """Gram-Schmidt orthogonalization to create orthonormal basis"""
    # First vector - just normalize
    u1 = v1.normalize()
    
    # Second vector - remove component along u1
    u2 = v2 - u1 * v2.dot(u1)
    u2 = u2.normalize()
    
    # Third vector - remove components along u1 and u2
    u3 = v3 - u1 * v3.dot(u1) - u2 * v3.dot(u2)
    u3 = u3.normalize()
    
    return u1, u2, u3

# Test Gram-Schmidt
v1 = Vec3(1, 0, 0)
v2 = Vec3(1, 1, 0)
v3 = Vec3(1, 1, 1)

u1, u2, u3 = gram_schmidt(v1, v2, v3)
print("\nGram-Schmidt Orthogonalization:")
print(f"u1 = {u1}")
print(f"u2 = {u2}")
print(f"u3 = {u3}")
print(f"u1 · u2 = {u1.dot(u2):.6f} (should be ~0)")
print(f"u1 · u3 = {u1.dot(u3):.6f} (should be ~0)")
print(f"u2 · u3 = {u2.dot(u3):.6f} (should be ~0)")

---

## 2. Matrix Implementation

In [ ]:
class Mat4:
    """4x4 Matrix for 3D transformations (homogeneous coordinates)"""
    
    def __init__(self, values=None):
        if values is None:
            # Identity matrix
            self.m = [[1 if i == j else 0 for j in range(4)] for i in range(4)]
        else:
            self.m = [list(row) for row in values]
    
    @staticmethod
    def identity():
        """Create identity matrix"""
        return Mat4()
    
    @staticmethod
    def zeros():
        """Create zero matrix"""
        return Mat4([[0]*4 for _ in range(4)])
    
    def __getitem__(self, index: int) -> list:
        return self.m[index]
    
    def __setitem__(self, index: int, value: list):
        self.m[index] = value
    
    # Matrix multiplication
    def __matmul__(self, other: 'Mat4') -> 'Mat4':
        result = Mat4.zeros()
        for i in range(4):
            for j in range(4):
                result[i][j] = sum(self.m[i][k] * other.m[k][j] for k in range(4))
        return result
    
    # Matrix-vector multiplication
    def mul_vec3(self, v: Vec3, w: float = 1.0) -> Vec3:
        """Multiply matrix by Vec3 (treating as homogeneous coordinate with w)"""
        x = self.m[0][0]*v.x + self.m[0][1]*v.y + self.m[0][2]*v.z + self.m[0][3]*w
        y = self.m[1][0]*v.x + self.m[1][1]*v.y + self.m[1][2]*v.z + self.m[1][3]*w
        z = self.m[2][0]*v.x + self.m[2][1]*v.y + self.m[2][2]*v.z + self.m[2][3]*w
        w_out = self.m[3][0]*v.x + self.m[3][1]*v.y + self.m[3][2]*v.z + self.m[3][3]*w
        
        # Perspective divide if w != 1
        if w_out != 0 and w_out != 1:
            return Vec3(x/w_out, y/w_out, z/w_out)
        return Vec3(x, y, z)
    
    # Transpose
    def transpose(self) -> 'Mat4':
        result = Mat4.zeros()
        for i in range(4):
            for j in range(4):
                result[i][j] = self.m[j][i]
        return result
    
    # Determinant (for 4x4)
    def det(self) -> float:
        """Calculate determinant using cofactor expansion"""
        m = self.m
        # Using first row cofactor expansion
        det = 0
        for j in range(4):
            det += ((-1)**j) * m[0][j] * self._minor(0, j)
        return det
    
    def _minor(self, row: int, col: int) -> float:
        """Calculate minor (3x3 determinant)"""
        sub = [[self.m[i][j] for j in range(4) if j != col] 
               for i in range(4) if i != row]
        return (sub[0][0] * (sub[1][1]*sub[2][2] - sub[1][2]*sub[2][1]) -
                sub[0][1] * (sub[1][0]*sub[2][2] - sub[1][2]*sub[2][0]) +
                sub[0][2] * (sub[1][0]*sub[2][1] - sub[1][1]*sub[2][0]))
    
    def __repr__(self) -> str:
        lines = ["Mat4(["]
        for row in self.m:
            lines.append("  [" + ", ".join(f"{x:7.3f}" for x in row) + "]")
        lines.append("])")
        return "\n".join(lines)

# Test Mat4
m = Mat4.identity()
print("Identity Matrix:")
print(m)

---

## 3. Transformation Matrices

In [ ]:
def translation_matrix(tx: float, ty: float, tz: float) -> Mat4:
    """Create translation matrix"""
    m = Mat4.identity()
    m[0][3] = tx
    m[1][3] = ty
    m[2][3] = tz
    return m

def scale_matrix(sx: float, sy: float, sz: float) -> Mat4:
    """Create scale matrix"""
    m = Mat4.zeros()
    m[0][0] = sx
    m[1][1] = sy
    m[2][2] = sz
    m[3][3] = 1.0
    return m

def rotation_x(angle: float) -> Mat4:
    """Rotation around X-axis (angle in radians)"""
    c = math.cos(angle)
    s = math.sin(angle)
    m = Mat4.identity()
    m[1][1] = c
    m[1][2] = -s
    m[2][1] = s
    m[2][2] = c
    return m

def rotation_y(angle: float) -> Mat4:
    """Rotation around Y-axis (angle in radians)"""
    c = math.cos(angle)
    s = math.sin(angle)
    m = Mat4.identity()
    m[0][0] = c
    m[0][2] = s
    m[2][0] = -s
    m[2][2] = c
    return m

def rotation_z(angle: float) -> Mat4:
    """Rotation around Z-axis (angle in radians)"""
    c = math.cos(angle)
    s = math.sin(angle)
    m = Mat4.identity()
    m[0][0] = c
    m[0][1] = -s
    m[1][0] = s
    m[1][1] = c
    return m

def rotation_axis_angle(axis: Vec3, angle: float) -> Mat4:
    """Rotation around arbitrary axis using Rodrigues' formula"""
    # Normalize axis
    axis = axis.normalize()
    c = math.cos(angle)
    s = math.sin(angle)
    t = 1 - c
    
    x, y, z = axis.x, axis.y, axis.z
    
    m = Mat4([
        [t*x*x + c,   t*x*y - s*z, t*x*z + s*y, 0],
        [t*x*y + s*z, t*y*y + c,   t*y*z - s*x, 0],
        [t*x*z - s*y, t*y*z + s*x, t*z*z + c,   0],
        [0,           0,           0,           1]
    ])
    return m

# Test transformations
print("Translation by (1, 2, 3):")
print(translation_matrix(1, 2, 3))

print("\nRotation around Z-axis by 90°:")
print(rotation_z(math.pi/2))

# Test composition
v = Vec3(1, 0, 0)
rot = rotation_z(math.pi/2)
v_rotated = rot.mul_vec3(v)
print(f"\nRotate (1,0,0) by 90° around Z: {v_rotated}")

---

## 4. Quaternions

In [ ]:
class Quaternion:
    """Quaternion for rotation representation"""
    
    def __init__(self, w: float = 1.0, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.w = w
        self.x = x
        self.y = y
        self.z = z
    
    @staticmethod
    def from_axis_angle(axis: Vec3, angle: float) -> 'Quaternion':
        """Create quaternion from axis-angle representation"""
        axis = axis.normalize()
        half_angle = angle / 2.0
        s = math.sin(half_angle)
        return Quaternion(
            math.cos(half_angle),
            axis.x * s,
            axis.y * s,
            axis.z * s
        )
    
    @staticmethod
    def from_euler(roll: float, pitch: float, yaw: float) -> 'Quaternion':
        """Create quaternion from Euler angles (in radians)"""
        cy = math.cos(yaw * 0.5)
        sy = math.sin(yaw * 0.5)
        cp = math.cos(pitch * 0.5)
        sp = math.sin(pitch * 0.5)
        cr = math.cos(roll * 0.5)
        sr = math.sin(roll * 0.5)
        
        return Quaternion(
            cr * cp * cy + sr * sp * sy,
            sr * cp * cy - cr * sp * sy,
            cr * sp * cy + sr * cp * sy,
            cr * cp * sy - sr * sp * cy
        )
    
    def normalize(self) -> 'Quaternion':
        """Return normalized quaternion"""
        norm = math.sqrt(self.w**2 + self.x**2 + self.y**2 + self.z**2)
        if norm > 0:
            return Quaternion(self.w/norm, self.x/norm, self.y/norm, self.z/norm)
        return Quaternion()
    
    def conjugate(self) -> 'Quaternion':
        """Return conjugate quaternion"""
        return Quaternion(self.w, -self.x, -self.y, -self.z)
    
    def __mul__(self, other: 'Quaternion') -> 'Quaternion':
        """Quaternion multiplication"""
        return Quaternion(
            self.w*other.w - self.x*other.x - self.y*other.y - self.z*other.z,
            self.w*other.x + self.x*other.w + self.y*other.z - self.z*other.y,
            self.w*other.y - self.x*other.z + self.y*other.w + self.z*other.x,
            self.w*other.z + self.x*other.y - self.y*other.x + self.z*other.w
        )
    
    def rotate_vector(self, v: Vec3) -> Vec3:
        """Rotate a vector using this quaternion"""
        # Convert vector to quaternion
        v_quat = Quaternion(0, v.x, v.y, v.z)
        # Rotate: q * v * q_conjugate
        result = self * v_quat * self.conjugate()
        return Vec3(result.x, result.y, result.z)
    
    def to_matrix(self) -> Mat4:
        """Convert quaternion to rotation matrix"""
        q = self.normalize()
        w, x, y, z = q.w, q.x, q.y, q.z
        
        return Mat4([
            [1-2*(y*y+z*z), 2*(x*y-w*z),   2*(x*z+w*y),   0],
            [2*(x*y+w*z),   1-2*(x*x+z*z), 2*(y*z-w*x),   0],
            [2*(x*z-w*y),   2*(y*z+w*x),   1-2*(x*x+y*y), 0],
            [0,             0,             0,             1]
        ])
    
    def __repr__(self) -> str:
        return f"Quat(w={self.w:.4f}, x={self.x:.4f}, y={self.y:.4f}, z={self.z:.4f})"

def slerp(q1: Quaternion, q2: Quaternion, t: float) -> Quaternion:
    """Spherical linear interpolation between two quaternions"""
    # Normalize quaternions
    q1 = q1.normalize()
    q2 = q2.normalize()
    
    # Compute dot product
    dot = q1.w*q2.w + q1.x*q2.x + q1.y*q2.y + q1.z*q2.z
    
    # If dot < 0, negate one quaternion to take shorter path
    if dot < 0:
        q2 = Quaternion(-q2.w, -q2.x, -q2.y, -q2.z)
        dot = -dot
    
    # Clamp dot to valid range
    dot = max(-1.0, min(1.0, dot))
    
    # If quaternions are very close, use linear interpolation
    if dot > 0.9995:
        result = Quaternion(
            q1.w + t * (q2.w - q1.w),
            q1.x + t * (q2.x - q1.x),
            q1.y + t * (q2.y - q1.y),
            q1.z + t * (q2.z - q1.z)
        )
        return result.normalize()
    
    # Perform SLERP
    theta = math.acos(dot)
    sin_theta = math.sin(theta)
    
    w1 = math.sin((1-t) * theta) / sin_theta
    w2 = math.sin(t * theta) / sin_theta
    
    return Quaternion(
        q1.w*w1 + q2.w*w2,
        q1.x*w1 + q2.x*w2,
        q1.y*w1 + q2.y*w2,
        q1.z*w1 + q2.z*w2
    )

# Test Quaternions
print("Quaternion Tests:")
axis = Vec3(0, 0, 1)
angle = math.pi / 2  # 90 degrees
q = Quaternion.from_axis_angle(axis, angle)
print(f"Quaternion from axis-angle: {q}")

v = Vec3(1, 0, 0)
v_rotated = q.rotate_vector(v)
print(f"Rotate (1,0,0) by 90° around Z: {v_rotated}")

# Test SLERP
q1 = Quaternion.from_axis_angle(Vec3(0, 0, 1), 0)
q2 = Quaternion.from_axis_angle(Vec3(0, 0, 1), math.pi/2)
q_mid = slerp(q1, q2, 0.5)
print(f"\nSLERP halfway: {q_mid}")
print(f"Expected angle: {math.pi/4:.4f} radians (45°)")

---

## 5. Visualization

In [ ]:
def plot_vectors_3d(vectors: list, labels: list = None, colors: list = None):
    """Plot 3D vectors"""
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    if labels is None:
        labels = [f"v{i}" for i in range(len(vectors))]
    if colors is None:
        colors = plt.cm.rainbow(np.linspace(0, 1, len(vectors)))
    
    for v, label, color in zip(vectors, labels, colors):
        ax.quiver(0, 0, 0, v.x, v.y, v.z, 
                 color=color, arrow_length_ratio=0.1, linewidth=2, label=label)
    
    # Set limits
    max_val = max(max(abs(v.x), abs(v.y), abs(v.z)) for v in vectors) * 1.2
    ax.set_xlim([-max_val, max_val])
    ax.set_ylim([-max_val, max_val])
    ax.set_zlim([-max_val, max_val])
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.title('3D Vector Visualization')
    plt.show()

# Visualize some vectors
v1 = Vec3(1, 0, 0)
v2 = Vec3(0, 1, 0)
v3 = Vec3(0, 0, 1)
v_cross = v1.cross(v2)

plot_vectors_3d(
    [v1, v2, v3, v_cross],
    ['X-axis', 'Y-axis', 'Z-axis', 'X × Y'],
    ['red', 'green', 'blue', 'purple']
)

---

## 6. Exercises

Try implementing these on your own:

1. **Vec2 and Vec4**: Create Vec2 and Vec4 classes similar to Vec3

2. **Matrix Inverse**: Implement matrix inversion for Mat4 using Gauss-Jordan elimination

3. **LookAt Matrix**: Implement a camera "look-at" transformation matrix:
   ```python
   def look_at(eye: Vec3, target: Vec3, up: Vec3) -> Mat4:
       # Create view matrix
       pass
   ```

4. **Perspective Projection**: Create a perspective projection matrix:
   ```python
   def perspective(fov: float, aspect: float, near: float, far: float) -> Mat4:
       # Create perspective projection
       pass
   ```

5. **Euler to Quaternion**: Create a function to convert Euler angles to quaternion (already partially done)

6. **Matrix Decomposition**: Decompose a transformation matrix into translation, rotation, and scale components

In [ ]:
# Your exercise implementations here


---

## Summary

In this chapter, you learned to implement:

✅ **Vec3** - Complete 3D vector class with all operations  
✅ **Mat4** - 4×4 matrix for transformations  
✅ **Transformations** - Translation, rotation, scale matrices  
✅ **Quaternions** - Rotation representation and SLERP  
✅ **Gram-Schmidt** - Orthonormalization algorithm  

These math primitives form the foundation for all 3D graphics operations!

**Next Chapter:** Framebuffer and 2D Rasterization